In [ ]:
import gym
from gym.spaces import Discrete, Box

import numpy as np

In [ ]:
def calc_reward(items, state, max_weight, burst_reward):
    reward = 0
    weight = 0
    
    for i in range(len(state)):
        reward += items[i][0] * state[i]
        weight += items[i][1] * state[i]
    
    if weight > max_weight or min(state) < 0:
        reward = burst_reward
    
    return reward, weight

# Env

In [ ]:
class Knapsack(gym.Env):
    def __init__(self, config):
        self.items = config["items"]
        self.max_weight = config["max_weight"]
        self.burst_reward = config["burst_reward"]
        
        h = self.max_weight // min(np.array(self.items)[:, 1])
        
        self.action_space = Box(0, h, shape = (len(self.items), ))
        self.observation_space = Box(0, h, shape = (len(self.items), ))

    def reset(self):
        return [0 for _ in self.items]

    def step(self, action):
        state = [round(d) for d in action]
        
        reward, _ = calc_reward(self.items, state, self.max_weight, self.burst_reward)
        
        return state, reward, True, {}

In [ ]:
items = [
    [120, 10],
    [130, 12],
    [80, 7],
    [100, 9],
    [250, 21],
    [185, 16]
]


In [ ]:
config = {
    "env": Knapsack, 
    "env_config": {"items": items, "max_weight": 65, "burst_reward": -100}
}


In [ ]:
import ray

#ray.shutdown()
ray.init()

In [ ]:
from ray.rllib.agents.ppo import PPOTrainer

trainer = PPOTrainer(config = config)

In [ ]:
from ray.rllib.agents.ddpg import DDPGTrainer

trainer = DDPGTrainer(config = config)

# Train

In [ ]:
from ray.tune.logger import pretty_print

for _ in range(20):
    r = trainer.train()
    print(pretty_print(r))


# Evaluate

In [ ]:
import collections

rs = []

for _ in range(1000):
    s = [0 for _ in range(len(items))]
    a = trainer.compute_action(s)
    
    s = [round(d) for d in a]
    
    r, w = calc_reward(items, s, 65, -1)
    
    rs.append(r)
    
    print(f"{s}, {r}, {w}")

collections.Counter(rs)

# Save

In [ ]:
checkpoint = trainer.save()
checkpoint

# Load

In [ ]:
trainer.restore(checkpoint)
